#### Задача 1 (10 баллов). 

Попробуем себя в решении задачи определения темы текста. Будем считать, что два текста похожи по теме, если у них больше общих слов (только не предлогов с союзами), чем у других текстов. У нашей программы для определения темы будет несколько готовых текстов (достаточно больших!) с уже известной темой в базе: выберите тексты (и темы) самостоятельно, 5-6 будет достаточно. 

Что должна делать программа? При запуске вы ей сообщаете название нового файла с текстом, который нужно классифицировать, она его открывает, обрабатывает и сравнивает с текстами в своей базе. С которым из текстов оказалось больше всего общих слов, того и тема! Очевидно, вам понадобится какие-то слова из текстов отбрасывать (подумайте, каким образом это сделать - здесь на самом деле несколько вариантов концепций), а еще лемматизировать или хотя бы использовать стемминг. 

Когда будете сдавать это задание, пожалуйста, пришлите и файлы с текстами. И имейте в виду, если тексты будут вставлены прямо в код и слишком короткие, я задачу засчитаю только вполовину. 

Напоминаю, как открываются файлы:

```
with open('путь к файлу - пишите прямые слеши', 'r', encoding='utf-8') as f:
    text = f.read() # все содержимое вашего файла считается в одну длинную строку
```

Настоятельно советую оформить код хотя бы в функции. 

In [22]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    words = [stemmer.stem(word) for word in words if word not in stopwords]
    return set(words)

def compare_texts(input_text, base_texts):
    common_words_count = {}

    for theme, base_text in base_texts.items():
        common_words = input_text.intersection(base_text)
        common_words_count[theme] = len(common_words)

    best_match_theme = max(common_words_count, key=common_words_count.get)
    return best_match_theme


stopwords = set(stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

file_paths = {
    'Технологии': r'тексты\technology.txt',
    'Спорт': r'тексты\sports.txt',
    'Путешествия': r'тексты\travel.txt',
    'История': r'тексты\history.txt',
    'Кулинария': r'тексты\cooking.txt'
}

texts = {theme: read_file(path) for theme, path in file_paths.items()}
processed_texts = {theme: preprocess_text(text) for theme, text in texts.items()}

input_text = read_file(r'тексты\input_history.txt')
processed_input = preprocess_text(input_text)
classified_theme = compare_texts(processed_input, processed_texts)
classified_theme

'История'

#### Задача 2 (10 баллов). 

Некоторые предлоги в русском языке могут управлять разными падежами (например, "я еду в Лондон" vs "я живу в Лондоне"). Давайте проанализируем эти предлоги и их падежи. Необходимо:

- составить список таких предлогов (РГ-80 вам в помощь)
- взять достаточно большой текст (можно большое художественное произведение)
- сделать морфоразбор этого текста
- Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога.

Примечания: во-первых, имейте в виду, что иногда после предлога могут идти самые неожиданные вещи: "я что, должен ехать на, черт побери, северный полюс?". Во-вторых, неплохо бы учитывать отсутствие пунктуации (конечно, в норме, как нам кажется, предлог обязательно требует зависимое, но! "да иди ты на!") Эти штуки можно отсеять, если просто учитывать только заранее определенные падежи, а не считать все, какие встретились (так и None можно огрести). 

Если будете использовать RNNMorph, возможно, понадобится регулярное выражение и немного терпения. 

In [58]:
import pymorphy2
from nltk.tokenize import word_tokenize
from collections import defaultdict


with open('тексты\crime_and_punishment.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

morph = pymorphy2.MorphAnalyzer()
tokens = word_tokenize(text, language='russian')

# Словарь предлогов и соответствующих падежей
prepositions_cases = {
    'в': ['accs', 'loct'], 'за': ['accs', 'ablt'], 'между': ['gent', 'ablt'],
    'на': ['accs', 'loct'], 'о': ['accs', 'loct'], 'под': ['accs', 'ablt'],
    'по': ['datv', 'accs', 'loct'], 'с': ['gent', 'accs', 'ablt']
}

# Словарь для подсчета падежей после предлогов
case_count = defaultdict(lambda: defaultdict(int))

# Анализ падежей слов после предлогов
for i, word in enumerate(tokens):
    if word.lower() in prepositions_cases:
        next_word = tokens[i + 1]
        parsed_word = morph.parse(next_word)[0]
        case = parsed_word.tag.case
        if case in prepositions_cases[word.lower()]:
            case_count[word.lower()][case] += 1
            
for prep, cases in case_count.items():
    print(f"Предлог '{prep}':")
    for case, count in cases.items():
        print(f'  {case}: {count} раз(а)')

Предлог 'с':
  ablt: 1385 раз(а)
  gent: 383 раз(а)
  accs: 89 раз(а)
Предлог 'в':
  loct: 1747 раз(а)
  accs: 1042 раз(а)
Предлог 'под':
  accs: 54 раз(а)
  ablt: 75 раз(а)
Предлог 'на':
  accs: 820 раз(а)
  loct: 492 раз(а)
Предлог 'о':
  loct: 258 раз(а)
  accs: 7 раз(а)
Предлог 'по':
  datv: 472 раз(а)
  loct: 84 раз(а)
  accs: 26 раз(а)
Предлог 'за':
  accs: 200 раз(а)
  ablt: 175 раз(а)
Предлог 'между':
  ablt: 59 раз(а)


#### Задача 3 (5 баллов). 

Представим, что у вас есть файл с разборами conllu (можете взять любой, например, [тут](https://github.com/dialogue-evaluation/GramEval2020)). Нужно просмотреть все примеры предложений с тегом dislocated и тегом discourse: напишите скрипт, который будет читать файл, находить все такие предложения и печатать: 1) сам текст предложения 2) слово, имеющее искомый тег. Если тег не был найден в файле, нужно об этом сообщить. Постарайтесь оформить вывод таким образом, чтобы это было удобно читать. 

In [1]:
def extract_sentences_with_tags(file_path):
    sentences_with_tags = []
    current_sentence_text = []
    sentence_with_tag = False
    tags_found = {'dislocated': False, 'discourse': False}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if not line.isspace():
                columns = line.split()
                word, tag = columns[1], columns[7]
                current_sentence_text.append(word)
                if tag in tags_found:
                    sentence_with_tag = True
                    tags_found[tag] = True
                    current_sentence_text[-1] = f"{word} ({tag})"
            else:   # Конец предложения
                if sentence_with_tag:
                    sentences_with_tags.append(' '.join(current_sentence_text))

                current_sentence_text = []
                sentence_with_tag = False

    tags_not_found = [tag for tag, found in tags_found.items() if not found]
    return sentences_with_tags, tags_not_found


sentences_with_tags, tags_not_found = extract_sentences_with_tags('GramEval2020-RuEval2017-social-dev.conllu')

formatted_sentences = '\n'.join(f'{i + 1}. {sentence}' for i, sentence in enumerate(sentences_with_tags))
print(formatted_sentences)

if tags_not_found:
    print('Следующие теги не были найдены:', ', '.join(tags_not_found))

1. Короче : столько - то (discourse) " я как Я " , остальное " я как МЫ " , а если только первое , то это — привет психические расстройства .
2. - Ой-ой-ой (discourse) , а сам - то (discourse) с мамой спишь каждый день , как маленький !
3. - Ну (discourse) ты подожди ещё , у неё и дети будут .
4. И они на ковре вертолете сваливают сначала на Ямайку , а потом в Бразилию , ну (discourse) это там где много диких обезьян в белых штанах .
Следующие теги не были найдены: dislocated


#### Задача 4 (5 баллов).

Возьмите любой достаточно длинный (лучше новостной) текст. Любым известным инструментом извлеките именованные сущности из этого текста и выведите их списком по категориям (т.е. персоны вместе, локации вместе, организации вместе). 

In [3]:
import spacy


with open(r'тексты\news.txt', 'r', encoding='utf-8') as f:
    news_text = f.read() 

nlp = spacy.load('ru_core_news_lg')
doc = nlp(news_text)

entities = {'PER': [], 'LOC': [], 'ORG': []}
for ent in doc.ents:
    entities[ent.label_].append(ent.text)

for category, entity_list in entities.items():
    print(f"{category}: {', '.join(entity_list)}")

PER: Стэнфорда, Лоуренс Саммерс, Клодин Гэй, Адам Гийетт, Гийетта, Хиллари Клинтон, Йинона Коэна, Коэн, Надя Абу Эль-Хадж, Абу Эль-Хадж, Манан Ахмед, Манан Ахмед, Лиз Мэгилл, Клодин Гэй, Росс Стивенс
LOC: Израиль, Газа, Нью-Йорка, Ближнем Востоке, Израиль, Газе, Палестиной, Израиль, Газе, Гарварда, Израиль, Нью-Йорке, США, Вашингтона, Израиль, Палестиной, Газа, США, Израилю, Палестине, Палестине, Палестины, Палестины, Южной Азии, Гарварда, Гарварда
ORG: Администрации, The New Yorker, ХАМАС, Колумбийском университете, Медуза, The New Yorker, СМИ, ХАМАС, ЦАХАЛ, Комитет солидарности студентов, Гарвардского университета, Гарварда, ХАМАС, Колумбийском университете, ХАМАС, Университета Колумбия, Американском университете, Колумбийского университета, Стэнфордском университете, Колумбийского университета, The New Yorker, Колумбийского университета, Колумбийского университета, The New Yorker, The Guardian, Колумбийского университета, Колумбийского университета, Колумбийского университета, The N

#### Задача на бонусные 5 баллов:

Сравните качество несколькиз разных морфопарсеров для любого языка, где их больше одного. Разберите этими морфопарсерами один и тот же текст, если они все разбирают в UD, можете вывести автоматически расхождения, в противном случае просмотрите глазами на наличие ошибок (текст, конечно, слишком большой лучше не брать). 

Без письменных выводов-комментариев не засчитывается. 

In [ ]:
# your code here